In [1]:
from tensorflow.keras.layers import Input, MaxPooling2D
from tensorflow.keras import Model
import numpy as np

In [2]:
inputs = Input(shape=(5,5,3))
x = MaxPooling2D(pool_size=2, padding='same')(inputs)
model = Model(inputs, x)

In [3]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5, 5, 3)]         0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 3, 3)          0         
 )                                                               
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [4]:
X = np.random.rand(1,5,5,3)
X

array([[[[0.9854169 , 0.80569586, 0.63268445],
         [0.96206047, 0.00886907, 0.67775967],
         [0.46792544, 0.81360698, 0.34777212],
         [0.90835439, 0.69174656, 0.98172619],
         [0.10715328, 0.91657229, 0.81624555]],

        [[0.57038264, 0.90280394, 0.77766336],
         [0.63409067, 0.82595812, 0.38151341],
         [0.79552045, 0.2163596 , 0.87461672],
         [0.74681147, 0.47584101, 0.93002867],
         [0.11400012, 0.97409111, 0.87404365]],

        [[0.53957961, 0.50863284, 0.89012595],
         [0.27801669, 0.55281537, 0.50004972],
         [0.31374426, 0.41849849, 0.81763357],
         [0.68099263, 0.21715725, 0.60245081],
         [0.23044868, 0.89402184, 0.73802917]],

        [[0.65681518, 0.45431134, 0.11186951],
         [0.63232732, 0.09914006, 0.469778  ],
         [0.53980327, 0.00999922, 0.32385368],
         [0.51189833, 0.23540886, 0.48827071],
         [0.97347711, 0.16329774, 0.73404494]],

        [[0.02010803, 0.73200388, 0.75296659],
     

In [5]:
y = model.predict(X)
y

1/1 [==============================] - 0s 42ms/step


2024-02-05 01:27:05.945781: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


array([[[[0.9854169 , 0.90280396, 0.77766335],
         [0.9083544 , 0.813607  , 0.98172617],
         [0.11400012, 0.9740911 , 0.87404364]],

        [[0.6568152 , 0.5528154 , 0.89012593],
         [0.6809926 , 0.4184985 , 0.81763357],
         [0.9734771 , 0.89402187, 0.7380292 ]],

        [[0.47080582, 0.89997756, 0.7775437 ],
         [0.8131852 , 0.86405236, 0.44136214],
         [0.5379043 , 0.45931736, 0.00951743]]]], dtype=float32)

In [6]:
X_in = [[[int(X[0][i][j][k]*1e36) for k in range(3)] for j in range(5)] for i in range(5)]

In [7]:
def MaxPooling2DInt(nRows, nCols, nChannels, poolSize, strides, input):
    out = [[[str(max(int(input[i*strides + x][j*strides + y][k]) for x in range(poolSize) for y in range(poolSize))) for k in range(nChannels)] for j in range((nCols - poolSize) // strides + 1)] for i in range((nRows - poolSize) // strides + 1)]
    return out

In [8]:
def MaxPooling2DsameInt(nRows, nCols, nChannels, poolSize, strides, input):
    if nRows % strides == 0:
        rowPadding = max(poolSize - strides, 0)
    else:
        rowPadding = max(poolSize - nRows % strides, 0)
    if nCols % strides == 0:
        colPadding = max(poolSize - strides, 0)
    else:
        colPadding = max(poolSize - nCols % strides, 0)
    
    _input = [[[0 for _ in range(nChannels)] for _ in range(nCols + colPadding)] for _ in range(nRows + rowPadding)]

    for i in range(nRows):
        for j in range(nCols):
            for k in range(nChannels):
                _input[i+rowPadding//2][j+colPadding//2][k] = input[i][j][k]
    
    out = MaxPooling2DInt(nRows + rowPadding, nCols + colPadding, nChannels, poolSize, strides, _input)
    return out

In [9]:
out = MaxPooling2DsameInt(5, 5, 3, 2, 2, X_in)
out

[[['985416895178787714183661703755988992',
   '902803940435826284139749932730941440',
   '777663362123877860598598645145665536'],
  ['908354388868282470618698257109352448',
   '813606982115283460844798537969434624',
   '981726193592448973649175667344932864'],
  ['114000116831430861403008392105033728',
   '974091109200621567612816335791194112',
   '874043654050301690820171495435665408']],
 [['656815180323522869339469446928924672',
   '552815374291668108366329210074038272',
   '890125946940297760199841005954924544'],
  ['680992631793690390926240128859897856',
   '418498494396639822766365224919367680',
   '817633569064628434285947918592507904'],
  ['973477114549060379932212627888930816',
   '894021844181003495121759207996522496',
   '738029169437394307130485433159385088']],
 [['470805829710577633964522992960536576',
   '899977578211573860664606514781093888',
   '777543729617406477245147183797239808'],
  ['813185240376988213714402965620523008',
   '864052363876363850452980196205133824',
  

In [10]:
in_json = {
    "in": X_in,
    "out": out
}

In [11]:
import json

In [12]:
with open("maxPooling2Dsame_input.json", "w") as f:
    json.dump(in_json, f)

In [13]:
inputs = Input(shape=(10,10,3))
x = MaxPooling2D(pool_size=2, strides=3, padding='same')(inputs)
model = Model(inputs, x)

In [14]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10, 10, 3)]       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 3)          0         
 2D)                                                             
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [15]:
X = np.random.rand(1,10,10,3)
X

array([[[[1.81060846e-01, 8.73335066e-01, 4.66591631e-01],
         [4.19512826e-01, 5.04687534e-01, 5.55108518e-01],
         [8.89727395e-01, 3.04715421e-01, 3.96868333e-01],
         [3.19583099e-01, 7.59947198e-04, 8.95376898e-01],
         [1.86082695e-01, 5.89361183e-01, 2.29939007e-01],
         [3.77869457e-01, 5.93831349e-01, 6.17898741e-01],
         [2.08890055e-01, 4.52999632e-02, 4.81015031e-02],
         [9.68292068e-01, 4.24571806e-01, 4.18124731e-02],
         [3.30703359e-02, 6.13358807e-01, 9.62525235e-01],
         [8.80909151e-01, 3.16377883e-02, 4.07229564e-01]],

        [[7.22047037e-01, 2.87769873e-01, 6.33057960e-01],
         [5.87593115e-01, 9.95928671e-01, 9.47646805e-02],
         [6.73764008e-01, 4.41940517e-01, 2.05872675e-01],
         [2.74105248e-01, 3.41288944e-01, 7.03928155e-01],
         [2.53407876e-01, 9.54086619e-01, 6.95949832e-01],
         [7.72974380e-01, 6.94646688e-01, 2.39040667e-01],
         [7.61119704e-01, 8.41368944e-01, 9.12971104e-

In [16]:
y = model.predict(X)
y

1/1 [==============================] - 0s 30ms/step


array([[[[0.72204703, 0.99592865, 0.63305795],
         [0.3195831 , 0.9540866 , 0.8953769 ],
         [0.96829206, 0.841369  , 0.9129711 ],
         [0.88090914, 0.7423906 , 0.7435633 ]],

        [[0.8272338 , 0.9749019 , 0.94703025],
         [0.71864825, 0.7759291 , 0.8370553 ],
         [0.9456178 , 0.9705187 , 0.6049204 ],
         [0.8657805 , 0.38088953, 0.74315953]],

        [[0.72634125, 0.94376737, 0.8796218 ],
         [0.5518502 , 0.6357877 , 0.5354464 ],
         [0.5874519 , 0.6902133 , 0.9091554 ],
         [0.70104164, 0.7786813 , 0.28307712]],

        [[0.51060206, 0.7244077 , 0.47525772],
         [0.9049647 , 0.90716904, 0.6091803 ],
         [0.39426044, 0.30080628, 0.85136044],
         [0.90124035, 0.373213  , 0.73612255]]]], dtype=float32)

In [17]:
X_in = [[[int(X[0][i][j][k]*1e36) for k in range(3)] for j in range(10)] for i in range(10)]

In [18]:
out = MaxPooling2DsameInt(10, 10, 3, 2, 3, X_in)
out

[[['722047037048510692294577429613117440',
   '995928671466629356695108803213918208',
   '633057959701190742109452742097371136'],
  ['319583098696415993297833594590330880',
   '954086618948581700566160227891675136',
   '895376897621437771937017335681908736'],
  ['968292067860118608616536728156504064',
   '841368944375512546814487235963912192',
   '912971104374087013640201200717529088'],
  ['880909150990983977881328708709515264',
   '742390579543848987849194909946871808',
   '743563277676321729157199064014520320']],
 [['827233800468269198331599375468331008',
   '974901899794306633179826070214410240',
   '947030261209948995604238219295588352'],
  ['718648242610498931670595372570378240',
   '775929100688812511565570103251566592',
   '837055315861977089490188905893330944'],
  ['945617766167062485510007332488085504',
   '970518691217475177799962810208223232',
   '604920400036749440823474048758448128'],
  ['865780466166194901147341462043623424',
   '380889549714929988856634904844173312',
   

In [19]:
in_json = {
    "in": X_in,
    "out": out
}

In [20]:
with open("maxPooling2Dsame_stride_input.json", "w") as f:
    json.dump(in_json, f)